<a href="https://colab.research.google.com/github/jongkwangyun/quant/blob/main/%EB%8F%99%EC%A0%81%EC%9E%90%EC%82%B0%EB%B0%B0%EB%B6%84_%EC%A0%84%ED%86%B5_%EB%93%80%EC%96%BC%EB%AA%A8%EB%A9%98%ED%85%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 첫 설치 이후 설치 안함
!pip install yfinance

In [ ]:
# 첫 설치 이후 설치 안함
!pip install pandas_market_calendars

In [ ]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import csv

# display에서 소수점 4자리까지 표현
pd.options.display.float_format = '{:.4f}'.format

# display시 DataFrame 모든 열 표시
pd.set_option('display.max_columns', None)

# 데이터 가져올 기간을 지정합니다. (오늘로부터 12개월 전)
today = datetime.today().date()
start_date = today - timedelta(days=375)

# 전략명
str_strategy = '전통 듀얼모멘텀 '

# 전략에 사용할 ETF 명 리스트에 담기
etf = ['SPY', 'EFA', 'BIL', 'AGG']

# 자산군 리스트 더하기, 중복 제거
etf_order = etf

# ETF 가격 추출
df_etf = yf.download(etf_order, start=start_date, end=today)['Adj Close']\
.sort_index(ascending=False)[etf_order].rename_axis(None, axis=0)

display(df_etf)

[*********************100%%**********************]  4 of 4 completed


,SPY,EFA,BIL,AGG
2023-08-25,439.9700,70.2300,91.7700,95.8700
2023-08-24,436.8900,69.7600,91.7500,95.9300
2023-08-23,443.0300,70.7300,91.7200,96.1500
2023-08-22,438.1500,69.9700,91.7000,95.2400
2023-08-21,439.3400,70.1000,91.6900,95.1400
...,...,...,...,...
2022-08-23,405.7246,62.2253,88.0619,98.6187
2022-08-22,406.7085,62.1960,88.0812,98.6576
2022-08-19,415.3573,63.2718,88.0716,99.0850
2022-08-18,421.0149,64.1521,88.0716,99.7845


In [ ]:
# csv 경로 설정
output_path =  '/content/drive/MyDrive/DynamicAssetAllocation/'

# 오늘 날짜를 가져와서 파일명에 추가
today_date = datetime.now().strftime('%y%m%d')
filename = f'{today_date} 전통 듀얼모멘텀.xlsx'

# 오늘 날짜 string에 저장
str_today = today.strftime('### %Y-%m-%d ')


# 월평균영업일수. average Monthly Business Day
MBD = 21

# 인덱스 만들기
df_momentum_index = ['today/' + str(df_etf.index[0].date()), '1mo_ago/' + str(df_etf.index[MBD].date()), \
                     '3mo_ago/' + str(df_etf.index[MBD*3].date()), '6mo_ago/' + str(df_etf.index[MBD*6].date()), \
                     '12mo_ago/' + str(df_etf.index[MBD*12].date())]


## DataFrame 만들기: 1-3-6-12 개월 etf 가격
price_today = df_etf.iloc[0:1]
price_1mo = df_etf.shift(-MBD).iloc[0:1]
price_3mo = df_etf.shift(-MBD*3).iloc[0:1]
price_6mo = df_etf.shift(-MBD*6).iloc[0:1]
price_12mo = df_etf.shift(-MBD*12).iloc[0:1]

df_price = pd.concat([price_today, price_1mo, price_3mo, price_6mo, price_12mo])
df_price.index = df_momentum_index

str_etf_price = str_today + str_strategy + "### ETF 현재-1-3-6-12 개월 가격"
display(Markdown(str_etf_price))
display(df_price)

# csv에 제목 및 DataFrame 추가. 드라이브 마운트 후 해야함
with open(output_path + filename, 'w') as f:
  f.write(f'{str_etf_price}\n')
  df_price.to_csv(f, index=True)

### 2023-08-27 전통 듀얼모멘텀 ### ETF 현재-1-3-6-12 개월 가격

,SPY,EFA,BIL,AGG
today/2023-08-25,439.9700,70.2300,91.7700,95.8700
1mo_ago/2023-07-27,452.4900,73.9100,91.3861,96.9164
3mo_ago/2023-05-25,413.1154,70.1540,90.5876,96.7186
6mo_ago/2023-02-24,393.4116,67.7972,89.5944,95.6477
12mo_ago/2022-08-24,407.0234,62.3035,88.0908,98.3564


In [ ]:
## DataFrame 만들기: 수익률
ror_1mo = df_etf.pct_change(-MBD).iloc[0:1]*100    # 1개월 수익률
ror_3mo = df_etf.pct_change(-MBD*3).iloc[0:1]*100    # 3개월 수익률
ror_6mo = df_etf.pct_change(-MBD*6).iloc[0:1]*100   # 6개월 수익률
ror_12mo = df_etf.pct_change(-MBD*12).iloc[0:1]*100  # 12개월 수익률

df_etf_ror = pd.concat([ror_1mo, ror_3mo, ror_6mo, ror_12mo])
df_etf_ror.index = df_momentum_index[1:]

str_etf_ror = str_today + str_strategy + "### ETF 1-3-6-12 개월 수익률"
display(Markdown(str_etf_ror))
display(df_etf_ror)

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_etf_ror}\n')
  df_etf_ror.to_csv(f, index=True)

### 2023-08-27 전통 듀얼모멘텀 ### ETF 1-3-6-12 개월 수익률

,SPY,EFA,BIL,AGG
1mo_ago/2023-07-27,-2.7669,-4.9790,0.4201,-1.0796
3mo_ago/2023-05-25,6.5005,0.1083,1.3052,-0.8774
6mo_ago/2023-02-24,11.8345,3.5883,2.4282,0.2324
12mo_ago/2022-08-24,8.0945,12.7223,4.1766,-2.5279


In [ ]:
display(Markdown(str_today[1:] + str_strategy))

# SPY, EFA 수익이 둘 다 BIL 보다 낮으면 AGG 에 투자
str_is_bond = 'False'
if ror_12mo['SPY'].iloc[0] < ror_12mo['BIL'].iloc[0] and ror_12mo['EFA'].iloc[0] < ror_12mo['BIL'].iloc[0]:
  str_is_bond = 'True'

str_etf = "## 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? " + str_is_bond
display(Markdown(str_etf))

# 아니라면 SPY, EFA 중 12개월 수익률이 더 높은 것 표시
str_higher_stock = ''
if ror_12mo['SPY'].iloc[0] > ror_12mo['EFA'].iloc[0]:
  str_higher_stock = 'SPY'
else:
  str_higher_stock = 'EFA'
str_higher_stock_text = "## 투자할 주식: " + str_higher_stock
display(Markdown(str_higher_stock_text))

# csv에 제목 및 DataFrame 추가
with open(output_path + filename, 'a') as f:
  f.write('\n')
  f.write(f'{str_today[1:] + str_strategy}\n')
  f.write(f'{str_etf}\n')
  f.write(f'{str_higher_stock_text}\n')

## 2023-08-27 전통 듀얼모멘텀 

## 미국 주식(SPY)과 글로벌 주식(EFA) 둘 다 초단기채(BIL) 보다 낮은가? False

## 투자할 주식: EFA